In [2]:
import pandas as pd
import numpy as np
import cv2 
import time
import pytesseract

from imutils.object_detection import non_max_suppression
from pytesseract import Output

In [3]:
WHITE = (255, 255, 255)
img = None
img0 = None
outputs = None

In [4]:
# Load names of classes and get random colors
classes = open('../models/coco.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')


In [5]:
net = cv2.dnn.readNetFromDarknet('../models/yolov3.cfg', '../models/yolov3.weights')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)

In [6]:
# determine the output layer
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [7]:
def load_image(path):
    global img, img0, outputs, ln

    img0 = cv2.imread(path)
    img = img0.copy()
    
    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    t0 = time.time()
    outputs = net.forward(ln)
    t = time.time() - t0

    # combine the 3 output groups into 1 (10647, 85)
    # large objects (507, 85)
    # medium objects (2028, 85)
    # small objects (8112, 85)
    outputs = np.vstack(outputs)

    post_process(img, outputs, 0.5)
    cv2.imshow('window',  img)
    #cv2.displayOverlay('window', f'forward propagation time={t:.3}')
    cv2.waitKey(0)

In [8]:
def post_process(img, outputs, conf):
    H, W = img.shape[:2]

    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        scores = output[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > conf:
            x, y, w, h = output[:4] * np.array([W, H, W, H])
            p0 = int(x - w//2), int(y - h//2)
            p1 = int(x + w//2), int(y + h//2)
            boxes.append([*p0, int(w), int(h)])
            confidences.append(float(confidence))
            classIDs.append(classID)
            # cv.rectangle(img, p0, p1, WHITE, 1)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            d = pytesseract.image_to_data(img[y:y+h, x:x+w], output_type=Output.DICT)
            print(d['text'])
            text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
            cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

In [9]:
def trackbar(x):
    global img
    conf = x/100
    img = img0.copy()
    post_process(img, outputs, conf)
    #cv2.displayOverlay('window', f'confidence level={conf}')
    cv2.imshow('window', img)


In [10]:
cv2.namedWindow('window')
cv2.createTrackbar('confidence', 'window', 50, 100, trackbar)

[ WARN:0] global /Users/runner/miniforge3/conda-bld/libopencv_1630689594685/work/modules/highgui/src/window.cpp (704) createTrackbar UI/Trackbar(confidence@window): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.


In [12]:
load_image('../data/IMG_0015.jpeg')

In [11]:
load_image('../data/IMG-20210924-WA0002.jpg')

error: OpenCV(4.5.3) /Users/runner/miniforge3/conda-bld/libopencv_1630689594685/work/modules/dnn/src/nms.cpp:36: error: (-215:Assertion failed) nms_threshold >= 0 in function 'NMSBoxes'


error: OpenCV(4.5.3) /Users/runner/miniforge3/conda-bld/libopencv_1630689594685/work/modules/dnn/src/nms.cpp:36: error: (-215:Assertion failed) nms_threshold >= 0 in function 'NMSBoxes'


error: OpenCV(4.5.3) /Users/runner/miniforge3/conda-bld/libopencv_1630689594685/work/modules/dnn/src/nms.cpp:36: error: (-215:Assertion failed) nms_threshold >= 0 in function 'NMSBoxes'


error: OpenCV(4.5.3) /Users/runner/miniforge3/conda-bld/libopencv_1630689594685/work/modules/dnn/src/nms.cpp:36: error: (-215:Assertion failed) nms_threshold >= 0 in function 'NMSBoxes'


error: OpenCV(4.5.3) /Users/runner/miniforge3/conda-bld/libopencv_1630689594685/work/modules/dnn/src/nms.cpp:36: error: (-215:Assertion failed) nms_threshold >= 0 in function 'NMSBoxes'


error: OpenCV(4.5.3) /Users/runner/miniforge3/conda-bld/libopencv_1630689594685/work/modules/dnn/src/nms.cpp:36: error: (-215:Assertion failed) nms_threshold >= 0 in function 'NMSBoxes'


error: OpenCV(4.5.3) /Users/runner/miniforge3/conda-bld/libopencv_1630689594685/work/modules/dnn/src/nms.cpp:36: error: (-215:Assertion failed) nms_threshold >= 0 in function 'NMSBoxes'


KeyboardInterrupt: 

In [ ]:
load_image('../data/IMG_0014.jpeg')